# Gabriel Borba - RM553187
# Enzo Teles de Moura - RM553899
# Pedro Henrique Mello Silva Alves - RM554223


# 


In [29]:
"""
SISTEMA FIREWATCH - COMBATE INTELIGENTE A QUEIMADAS FLORESTAIS
Sistema integrado de detecção, alerta e combate a queimadas usando IoT, drones e análise de dados

Componentes do Sistema:
- Sensores IoT: Monitoramento de temperatura, fumaça e umidade
- Drones Autônomos: Detecção térmica e combate inicial
- Plataforma Web/App: Interface de monitoramento e alertas
- Central de Emergências: Coordenação de equipes e recursos

Estruturas de Dados Implementadas:
- Fila (Queue): Processamento de dados dos sensores IoT
- Heap: Priorização de alertas por criticidade
- Pilha (Stack): Histórico de ações dos drones
- Lista Ligada: Gerenciamento dinâmico de sensores ativos
- Árvore: Hierarquia de zonas de monitoramento
- Grafo: Rede de sensores e otimização de rotas de drones
"""

import heapq
import json
import math
from collections import deque, defaultdict
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional

# ================================ FILA (QUEUE) -  ================================
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'

# Exemplo de uso:
print(f"{bcolors.OKGREEN}✅ Configuração concluída{bcolors.ENDC}")
class FilaDadosSensores:
    """
    Fila para processar dados dos sensores IoT em tempo real
    Os dados chegam continuamente dos sensores de campo via LoRaWAN
    """
    
    def __init__(self):
        self.dados_sensores = deque()  # FIFO para processamento sequencial
        self.contador_leituras = 1
        
    def adicionar_leitura_sensor(self, sensor_id: str, temperatura: float, 
                                fumaca: float, umidade: float, localizacao: Tuple[float, float]):
        """Adiciona nova leitura de sensor IoT na fila para processamento"""
        leitura = {
            "id_leitura": self.contador_leituras,
            "sensor_id": sensor_id,
            "temperatura": temperatura,
            "fumaca": fumaca,
            "umidade": umidade,
            "localizacao": localizacao,  # (latitude, longitude)
            "timestamp": datetime.now(),
            "processado": False
        }
        
        self.dados_sensores.append(leitura)
        self.contador_leituras += 1
        
        print(f"📡 Sensor {sensor_id}: T={temperatura}°C, Fumaça={fumaca}%, Umidade={umidade}%")
        return leitura
    
    def processar_proxima_leitura(self):
        """Processa próxima leitura da fila (FIFO)"""
        if self.dados_sensores:
            leitura = self.dados_sensores.popleft()
            leitura["processado"] = True
            return leitura
        return None
    
    def tamanho_fila(self):
        """Retorna quantidade de leituras pendentes"""
        return len(self.dados_sensores)
    
    def listar_pendentes(self):
        """Lista todas as leituras pendentes de processamento"""
        return list(self.dados_sensores)

# ================================ HEAP -  ================================
class HeapAlertasCriticos:
    """
    Heap para priorizar alertas de incêndio por criticidade
    Integra dados de múltiplos sensores para determinar urgência
    """
    
    def __init__(self):
        self.heap_alertas = []  # Min-heap (usamos valores negativos para max-heap)
        self.limiares = {
            "temperatura_critica": 45.0,      # °C
            "fumaca_critica": 30.0,           # %
            "umidade_baixa": 20.0,            # %
            "vento_alto": 25.0                # km/h
        }
        
    def calcular_indice_risco(self, leitura: dict, dados_clima: dict = None) -> float:
        """
        Calcula índice de risco de incêndio baseado em múltiplos fatores
        Fórmula: (Temp_norm + Fumaça_norm) * Fator_umidade * Fator_vento
        """
        # Normalização dos valores (0-1)
        temp_norm = min(leitura["temperatura"] / 50.0, 1.0)
        fumaca_norm = min(leitura["fumaca"] / 100.0, 1.0)
        
        # Fator de umidade (umidade baixa = risco alto)
        fator_umidade = max(0.5, (100 - leitura["umidade"]) / 100.0)
        
        # Fator de vento (se disponível)
        fator_vento = 1.0
        if dados_clima and "velocidade_vento" in dados_clima:
            fator_vento = 1.0 + (dados_clima["velocidade_vento"] / 50.0)
        
        # Cálculo final do índice
        indice = (temp_norm + fumaca_norm) * fator_umidade * fator_vento
        return min(indice, 10.0)  # Máximo 10.0
    
    def adicionar_alerta(self, leitura: dict, dados_clima: dict = None):
        """Adiciona alerta no heap baseado no índice de risco"""
        indice_risco = self.calcular_indice_risco(leitura, dados_clima)
        
        # Determina criticidade
        if indice_risco >= 7.0:
            criticidade = "CRÍTICO"
        elif indice_risco >= 5.0:
            criticidade = "ALTO"
        elif indice_risco >= 3.0:
            criticidade = "MÉDIO"
        else:
            criticidade = "BAIXO"
        
        alerta = {
            "sensor_id": leitura["sensor_id"],
            "localizacao": leitura["localizacao"],
            "indice_risco": indice_risco,
            "criticidade": criticidade,
            "timestamp": leitura["timestamp"],
            "dados_sensor": leitura
        }
        
        # Adiciona no heap (valor negativo para simular max-heap)
        heapq.heappush(self.heap_alertas, (-indice_risco, leitura["timestamp"], alerta))
        
        if indice_risco >= 5.0:
            print(f"🚨 ALERTA {criticidade}: Sensor {leitura['sensor_id']} - Risco {indice_risco:.1f}")
        
        return alerta
    
    def processar_alerta_mais_critico(self):
        """Remove e retorna o alerta mais crítico"""
        if self.heap_alertas:
            risco_neg, timestamp, alerta = heapq.heappop(self.heap_alertas)
            return alerta
        return None
    
    def listar_alertas_criticos(self, limite: int = 5):
        """Lista os alertas mais críticos sem removê-los"""
        alertas_ordenados = sorted(self.heap_alertas, key=lambda x: x[0])
        return [alerta[2] for alerta in alertas_ordenados[:limite]]

# ================================ PILHA (STACK) -  ================================
class PilhaAcoesDrone:
    """
    Pilha para registrar histórico de ações dos drones autônomos
    Cada drone mantém seu próprio histórico de missões
    """
    
    def __init__(self, drone_id: str):
        self.drone_id = drone_id
        self.historico_acoes = []  # Stack (LIFO)
        self.status_atual = "DISPONÍVEL"
        self.coordenadas_atual = (0.0, 0.0)
        self.nivel_combustivel = 100.0
        self.nivel_agua = 100.0
        
    def executar_missao(self, tipo_missao: str, localizacao: Tuple[float, float], 
                       dados_termicos: dict = None):
        """
        Executa missão e registra na pilha
        Tipos: RECONHECIMENTO, COMBATE, MONITORAMENTO, RESGATE
        """
        missao = {
            "id_missao": len(self.historico_acoes) + 1,
            "tipo": tipo_missao,
            "localizacao": localizacao,
            "timestamp_inicio": datetime.now(),
            "timestamp_fim": None,
            "dados_termicos": dados_termicos or {},
            "recursos_utilizados": {},
            "resultado": "EM_ANDAMENTO"
        }
        
        # Simula execução da missão
        if tipo_missao == "RECONHECIMENTO":
            missao["dados_termicos"] = self._simular_dados_termicos(localizacao)
            missao["recursos_utilizados"] = {"tempo_voo": 15, "combustivel": 10}
            
        elif tipo_missao == "COMBATE":
            agua_utilizada = min(self.nivel_agua, 50.0)
            missao["recursos_utilizados"] = {
                "agua_litros": agua_utilizada,
                "tempo_combate": 20,
                "combustivel": 25
            }
            self.nivel_agua -= agua_utilizada
            
        elif tipo_missao == "MONITORAMENTO":
            missao["recursos_utilizados"] = {"tempo_voo": 30, "combustivel": 15}
            
        # Atualiza status e posição
        self.coordenadas_atual = localizacao
        self.nivel_combustivel -= missao["recursos_utilizados"].get("combustivel", 0)
        self.status_atual = "EM_MISSAO"
        
        # Adiciona na pilha (LIFO)
        self.historico_acoes.append(missao)
        
        print(f"🚁 Drone {self.drone_id}: Missão {tipo_missao} iniciada em {localizacao}")
        return missao
    
    def finalizar_missao_atual(self, resultado: str = "SUCESSO", observacoes: str = ""):
        """Finaliza a missão atual (topo da pilha)"""
        if self.historico_acoes:
            missao_atual = self.historico_acoes[-1]  # Topo da pilha
            if missao_atual["resultado"] == "EM_ANDAMENTO":
                missao_atual["timestamp_fim"] = datetime.now()
                missao_atual["resultado"] = resultado
                missao_atual["observacoes"] = observacoes
                self.status_atual = "DISPONÍVEL"
                
                duracao = (missao_atual["timestamp_fim"] - missao_atual["timestamp_inicio"]).seconds
                print(f"✅ Drone {self.drone_id}: Missão finalizada - {resultado} ({duracao}s)")
                return missao_atual
        return None
    
    def abortar_missao_atual(self):
        """Aborta e remove a missão atual da pilha"""
        if self.historico_acoes:
            missao_abortada = self.historico_acoes.pop()  # Remove do topo
            self.status_atual = "DISPONÍVEL"
            print(f"❌ Drone {self.drone_id}: Missão abortada - {missao_abortada['tipo']}")
            return missao_abortada
        return None
    
    def obter_ultima_missao(self):
        """Retorna última missão sem remover da pilha"""
        return self.historico_acoes[-1] if self.historico_acoes else None
    
    def _simular_dados_termicos(self, localizacao: Tuple[float, float]) -> dict:
        """Simula dados da câmera térmica do drone"""
        return {
            "temperatura_max": 45.0 + (hash(str(localizacao)) % 20),
            "pontos_calor": hash(str(localizacao)) % 10,
            "area_afetada_m2": (hash(str(localizacao)) % 1000) * 10,
            "intensidade_fogo": ["BAIXA", "MÉDIA", "ALTA"][hash(str(localizacao)) % 3]
        }

# ================================ LISTA LIGADA - ================================
class NoSensor:
    """Nó para lista ligada de sensores IoT"""
    def __init__(self, sensor_id: str, tipo: str, localizacao: Tuple[float, float]):
        self.sensor_id = sensor_id
        self.tipo = tipo  # "TEMPERATURA", "FUMACA", "UMIDADE", "MULTISENSOR"
        self.localizacao = localizacao
        self.status = "ATIVO"
        self.ultima_leitura = None
        self.timestamp_instalacao = datetime.now()
        self.bateria = 100.0
        self.proximo = None

class ListaSensoresIoT:
    """
    Lista ligada para gerenciar sensores IoT dinamicamente
    Permite adicionar/remover sensores conforme necessário
    """
    
    def __init__(self):
        self.cabeca = None
        self.total_sensores = 0
        
    def instalar_sensor(self, sensor_id: str, tipo: str, localizacao: Tuple[float, float]):
        """Instala novo sensor na rede IoT"""
        novo_sensor = NoSensor(sensor_id, tipo, localizacao)
        novo_sensor.proximo = self.cabeca
        self.cabeca = novo_sensor
        self.total_sensores += 1
        
        print(f"📡 Sensor {sensor_id} ({tipo}) instalado em {localizacao}")
        return novo_sensor
    
    def atualizar_status_sensor(self, sensor_id: str, novo_status: str, nivel_bateria: float = None):
        """Atualiza status de um sensor específico"""
        atual = self.cabeca
        while atual:
            if atual.sensor_id == sensor_id:
                status_anterior = atual.status
                atual.status = novo_status
                if nivel_bateria is not None:
                    atual.bateria = nivel_bateria
                
                print(f"🔧 Sensor {sensor_id}: {status_anterior} → {novo_status}")
                return True
            atual = atual.proximo
        return False
    
    def buscar_sensor(self, sensor_id: str) -> Optional[NoSensor]:
        """Busca sensor específico na lista"""
        atual = self.cabeca
        while atual:
            if atual.sensor_id == sensor_id:
                return atual
            atual = atual.proximo
        return None
    
    def listar_sensores_por_status(self, status: str = None) -> List[dict]:
        """Lista sensores filtrados por status"""
        sensores = []
        atual = self.cabeca
        while atual:
            if status is None or atual.status == status:
                sensores.append({
                    "id": atual.sensor_id,
                    "tipo": atual.tipo,
                    "localizacao": atual.localizacao,
                    "status": atual.status,
                    "bateria": atual.bateria,
                    "instalacao": atual.timestamp_instalacao
                })
            atual = atual.proximo
        return sensores
    
    def remover_sensor_danificado(self, sensor_id: str):
        """Remove sensor danificado da rede"""
        if not self.cabeca:
            return False
            
        if self.cabeca.sensor_id == sensor_id:
            self.cabeca = self.cabeca.proximo
            self.total_sensores -= 1
            print(f"❌ Sensor {sensor_id} removido da rede")
            return True
        
        atual = self.cabeca
        while atual.proximo:
            if atual.proximo.sensor_id == sensor_id:
                atual.proximo = atual.proximo.proximo
                self.total_sensores -= 1
                print(f"❌ Sensor {sensor_id} removido da rede")
                return True
            atual = atual.proximo
        return False
    
    def obter_sensores_proximos(self, localizacao: Tuple[float, float], 
                               raio_km: float = 5.0) -> List[dict]:
        """Encontra sensores próximos a uma localização"""
        sensores_proximos = []
        atual = self.cabeca
        
        while atual:
            distancia = self._calcular_distancia(localizacao, atual.localizacao)
            if distancia <= raio_km:
                sensores_proximos.append({
                    "sensor": atual,
                    "distancia_km": distancia
                })
            atual = atual.proximo
        
        return sorted(sensores_proximos, key=lambda x: x["distancia_km"])
    
    def _calcular_distancia(self, loc1: Tuple[float, float], loc2: Tuple[float, float]) -> float:
        """Calcula distância entre duas coordenadas usando fórmula de Haversine"""
        lat1, lon1 = math.radians(loc1[0]), math.radians(loc1[1])
        lat2, lon2 = math.radians(loc2[0]), math.radians(loc2[1])
        
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
        c = 2 * math.asin(math.sqrt(a))
        
        return 6371 * c  # Raio da Terra em km

# ================================ ÁRVORE -  ================================
class NoZonaMonitoramento:
    """Nó para árvore de zonas de monitoramento"""
    def __init__(self, nome: str, tipo: str, nivel: int):
        self.nome = nome
        self.tipo = tipo  # "REGIAO", "ESTADO", "MUNICIPIO", "DISTRITO", "SETOR"
        self.nivel = nivel
        self.filhos = []
        self.pai = None
        self.sensores_ativos = []
        self.drones_alocados = []
        self.nivel_risco = "BAIXO"
        self.area_km2 = 0.0

class ArvoreZonasMonitoramento:
    """
    Árvore hierárquica de zonas de monitoramento
    Região → Estado → Município → Distrito → Setor
    """
    
    def __init__(self, nome_regiao: str = "Sudeste"):
        self.raiz = NoZonaMonitoramento(nome_regiao, "REGIAO", 0)
        self.mapa_zonas = {nome_regiao: self.raiz}
        print(f"🌳 Sistema de monitoramento iniciado para região: {nome_regiao}")
    
    def adicionar_estado(self, nome_estado: str, area_km2: float = 0.0):
        """Adiciona estado à região"""
        estado = NoZonaMonitoramento(nome_estado, "ESTADO", 1)
        estado.pai = self.raiz
        estado.area_km2 = area_km2
        self.raiz.filhos.append(estado)
        self.mapa_zonas[nome_estado] = estado
        
        print(f"🏛️ Estado {nome_estado} adicionado ({area_km2} km²)")
        return estado
    
    def adicionar_municipio(self, nome_estado: str, nome_municipio: str, area_km2: float = 0.0):
        """Adiciona município a um estado"""
        estado = self.mapa_zonas.get(nome_estado)
        if estado:
            municipio = NoZonaMonitoramento(nome_municipio, "MUNICIPIO", 2)
            municipio.pai = estado
            municipio.area_km2 = area_km2
            estado.filhos.append(municipio)
            self.mapa_zonas[f"{nome_estado}_{nome_municipio}"] = municipio
            
            print(f"🏘️ Município {nome_municipio} adicionado ao estado {nome_estado}")
            return municipio
        return None
    
    def adicionar_setor_monitoramento(self, nome_estado: str, nome_municipio: str, 
                                     nome_setor: str, area_km2: float = 0.0):
        """Adiciona setor de monitoramento específico"""
        municipio = self.mapa_zonas.get(f"{nome_estado}_{nome_municipio}")
        if municipio:
            setor = NoZonaMonitoramento(nome_setor, "SETOR", 3)
            setor.pai = municipio
            setor.area_km2 = area_km2
            municipio.filhos.append(setor)
            self.mapa_zonas[f"{nome_estado}_{nome_municipio}_{nome_setor}"] = setor
            
            print(f"🌲 Setor {nome_setor} criado em {nome_municipio}")
            return setor
        return None
    
    def alocar_drone_zona(self, caminho_zona: str, drone_id: str):
        """Aloca drone específico a uma zona"""
        zona = self.mapa_zonas.get(caminho_zona)
        if zona:
            zona.drones_alocados.append(drone_id)
            print(f"🚁 Drone {drone_id} alocado para {zona.nome}")
            return True
        return False
    
    def atualizar_nivel_risco_zona(self, caminho_zona: str, nivel_risco: str):
        """Atualiza nível de risco de uma zona específica"""
        zona = self.mapa_zonas.get(caminho_zona)
        if zona:
            zona.nivel_risco = nivel_risco
            print(f"⚠️ Zona {zona.nome}: Risco atualizado para {nivel_risco}")
            
            # Propaga risco para nível superior se necessário
            if nivel_risco in ["CRÍTICO", "ALTO"] and zona.pai:
                self._propagar_risco_hierarquia(zona.pai, nivel_risco)
            return True
        return False
    
    def _propagar_risco_hierarquia(self, zona_pai: NoZonaMonitoramento, nivel_risco: str):
        """Propaga risco para níveis superiores da hierarquia"""
        if zona_pai.nivel_risco == "BAIXO" and nivel_risco in ["ALTO", "CRÍTICO"]:
            zona_pai.nivel_risco = "MÉDIO"
            print(f"📈 Risco propagado para {zona_pai.nome}: {zona_pai.nivel_risco}")
    
    def visualizar_hierarquia_completa(self):
        """Visualiza toda a hierarquia de monitoramento"""
        print(f"\n🌳 HIERARQUIA DE MONITORAMENTO:")
        self._imprimir_zona(self.raiz, 0)
    
    def _imprimir_zona(self, zona: NoZonaMonitoramento, nivel: int):
        """Imprime zona e seus filhos recursivamente"""
        indent = "  " * nivel
        simbolo = "📍" if nivel == 0 else "├──" if nivel < 3 else "└──"
        
        info_extra = f" ({zona.area_km2} km²)" if zona.area_km2 > 0 else ""
        risco_emoji = {"BAIXO": "🟢", "MÉDIO": "🟡", "ALTO": "🟠", "CRÍTICO": "🔴"}
        
        print(f"{indent}{simbolo} {zona.nome} ({zona.tipo}){info_extra} {risco_emoji[zona.nivel_risco]}")
        
        if zona.drones_alocados:
            print(f"{indent}    🚁 Drones: {', '.join(zona.drones_alocados)}")
        if zona.sensores_ativos:
            print(f"{indent}    📡 Sensores: {len(zona.sensores_ativos)}")
        
        for filho in zona.filhos:
            self._imprimir_zona(filho, nivel + 1)

# ================================ GRAFO - ================================
class GrafoRedeSensores:
    """
    Grafo representando rede de sensores e otimização de rotas para drones
    Vértices: Sensores, Bases de Drones, Pontos de Interesse
    Arestas: Rotas de voo com custos (tempo, distância, combustível)
    """
    
    def __init__(self):
        self.grafo = defaultdict(dict)
        self.coordenadas = {}  # Mapeia IDs para coordenadas geográficas
        self.tipos_nos = {}    # Mapeia IDs para tipos (SENSOR, BASE, POI)
        
    def adicionar_no(self, no_id: str, coordenadas: Tuple[float, float], tipo: str):
        """
        Adiciona nó ao grafo (sensor, base de drone, ponto de interesse)
        """
        self.coordenadas[no_id] = coordenadas
        self.tipos_nos[no_id] = tipo
        
        if no_id not in self.grafo:
            self.grafo[no_id] = {}
            
        emoji_tipo = {"SENSOR": "📡", "BASE": "🏠", "POI": "📍"}
        print(f"{emoji_tipo.get(tipo, '❓')} Nó {no_id} ({tipo}) adicionado em {coordenadas}")
    
    def conectar_nos(self, origem: str, destino: str, peso_personalizado: float = None):
        """
        Conecta dois nós calculando peso baseado em distância e fatores de voo
        """
        if origem not in self.coordenadas or destino not in self.coordenadas:
            return False
        
        if peso_personalizado:
            peso = peso_personalizado
        else:
            # Calcula peso baseado em distância e fatores de voo
            distancia = self._calcular_distancia_nos(origem, destino)
            peso = self._calcular_peso_voo(origem, destino, distancia)
        
        # Conexão bidirecional
        self.grafo[origem][destino] = peso
        self.grafo[destino][origem] = peso
        
        print(f"🔗 Conexão: {origem} ↔ {destino} (peso: {peso:.1f})")
        return True
    
    def _calcular_distancia_nos(self, origem: str, destino: str) -> float:
        """Calcula distância euclidiana entre dois nós"""
        coord1 = self.coordenadas[origem]
        coord2 = self.coordenadas[destino]
        
        return math.sqrt((coord2[0] - coord1[0])**2 + (coord2[1] - coord1[1])**2) * 111  # Aprox km
    
    def _calcular_peso_voo(self, origem: str, destino: str, distancia: float) -> float:
        """
        Calcula peso de voo considerando fatores como:
        - Distância base
        - Tipo de terreno
        - Fatores meteorológicos
        """
        peso_base = distancia
        
        # Fatores de correção baseados no tipo de nó
        tipo_origem = self.tipos_nos[origem]
        tipo_destino = self.tipos_nos[destino]
        
        # Voo de/para base tem menor custo (infraestrutura)
        if tipo_origem == "BASE" or tipo_destino == "BASE":
            peso_base *= 0.9
        
        # Voo para sensor em emergência tem prioridade
        if tipo_destino == "SENSOR":
            peso_base *= 0.8
        
        return peso_base
    
    def dijkstra_otimizado(self, origem: str, destino: str) -> Tuple[List[str], float]:
        """
        Algoritmo de Dijkstra otimizado para rotas de drone
        Retorna: (caminho_otimo, custo_total)
        """
        if origem not in self.grafo or destino not in self.grafo:
            return [], float('inf')
        
        # Inicialização
        distancias = {no: float('inf') for no in self.grafo}
        distancias[origem] = 0
        predecessores = {}
        nao_visitados = set(self.grafo.keys())
        
        while nao_visitados:
            # Encontra nó não visitado com menor distância
            atual = min(nao_visitados, key=lambda x: distancias[x])
            
            if distancias[atual] == float('inf'):
                break
                
            nao_visitados.remove(atual)
            
            # Relaxamento das arestas
            for vizinho, peso in self.grafo[atual].items():
                if vizinho in nao_visitados:
                    nova_distancia = distancias[atual] + peso
                    if nova_distancia < distancias[vizinho]:
                        distancias[vizinho] = nova_distancia
                        predecessores[vizinho] = atual
        
        # Reconstrói caminho
        if destino not in predecessores and origem != destino:
            return [], float('inf')
        
        caminho = []
        atual = destino
        while atual is not None:
            caminho.append(atual)
            atual = predecessores.get(atual)
        
        caminho.reverse()
        return caminho, distancias[destino]
    
    def encontrar_rota_otima_drone(self, base_drone: str, local_emergencia: str) -> dict:
        """
        Encontra rota ótima para drone responder a emergência
        Considera sensores intermediários para coleta de dados
        """
        # Encontra rota direta
        rota_direta, custo_direto = self.dijkstra_otimizado(base_drone, local_emergencia)
        
        # Encontra sensores próximos ao local de emergência
        sensores_proximos = self._encontrar_sensores_proximos_emergencia(local_emergencia, 2.0)
        
        # Testa rota passando por sensores intermediários
        melhor_rota = rota_direta
        melhor_custo = custo_direto
        dados_coletados = []
        
        for sensor in sensores_proximos[:2]:  # Máximo 2 sensores intermediários
            rota_via_sensor, custo_via_sensor = self._calcular_rota_via_sensor(
                base_drone, sensor["sensor_id"], local_emergencia
            )
            
            # Se rota via sensor for mais eficiente (considerando dados coletados)
            if custo_via_sensor < melhor_custo * 1.2:  # Aceita até 20% mais custo pelos dados
                melhor_rota = rota_via_sensor
                melhor_custo = custo_via_sensor
                dados_coletados.append(sensor["sensor_id"])
        
        resultado = {
            "rota_otima": melhor_rota,
            "custo_total": melhor_custo,
            "base_origem": base_drone,
            "destino": local_emergencia,
            "sensores_intermediarios": dados_coletados,
            "tempo_estimado_min": melhor_custo / 0.5  # Assume velocidade média 30km/h
        }
        
        print(f"🗺️ Rota calculada: {' → '.join(melhor_rota)} (Custo: {melhor_custo:.1f})")
        return resultado
    
    def _encontrar_sensores_proximos_emergencia(self, local: str, raio_km: float) -> List[dict]:
        """Encontra sensores próximos ao local de emergência"""
        if local not in self.coordenadas:
            return []
        
        coord_emergencia = self.coordenadas[local]
        sensores_proximos = []
        
        for no_id, tipo in self.tipos_nos.items():
            if tipo == "SENSOR":
                distancia = self._calcular_distancia_nos(local, no_id)
                if distancia <= raio_km:
                    sensores_proximos.append({
                        "sensor_id": no_id,
                        "distancia": distancia,
                        "coordenadas": self.coordenadas[no_id]
                    })
        
        return sorted(sensores_proximos, key=lambda x: x["distancia"])
    
    def _calcular_rota_via_sensor(self, origem: str, sensor: str, destino: str) -> Tuple[List[str], float]:
        """Calcula rota passando por sensor intermediário"""
        rota1, custo1 = self.dijkstra_otimizado(origem, sensor)
        rota2, custo2 = self.dijkstra_otimizado(sensor, destino)
        
        if rota1 and rota2:
            # Remove duplicata do sensor na junção das rotas
            rota_completa = rota1 + rota2[1:]
            custo_total = custo1 + custo2
            return rota_completa, custo_total
        
        return [], float('inf')
    
    def analisar_conectividade_rede(self) -> dict:
        """Analisa conectividade e robustez da rede de sensores"""
        total_nos = len(self.grafo)
        total_arestas = sum(len(vizinhos) for vizinhos in self.grafo.values()) // 2
        
        # Encontra componentes conectados
        componentes = self._encontrar_componentes_conectados()
        
        # Identifica pontos críticos (nós cuja remoção fragmentaria a rede)
        pontos_criticos = self._identificar_pontos_criticos()
        
        analise = {
            "total_nos": total_nos,
            "total_arestas": total_arestas,
            "componentes_conectados": len(componentes),
            "maior_componente": max(len(comp) for comp in componentes) if componentes else 0,
            "pontos_criticos": pontos_criticos,
            "densidade_rede": (2 * total_arestas) / (total_nos * (total_nos - 1)) if total_nos > 1 else 0
        }
        
        print(f"📊 Análise da Rede: {total_nos} nós, {total_arestas} conexões")
        print(f"🔗 Componentes conectados: {len(componentes)}")
        
        return analise
    
    def _encontrar_componentes_conectados(self) -> List[List[str]]:
        """Encontra componentes conectados usando DFS"""
        visitados = set()
        componentes = []
        
        for no in self.grafo:
            if no not in visitados:
                componente = []
                self._dfs_componente(no, visitados, componente)
                componentes.append(componente)
        
        return componentes
    
    def _dfs_componente(self, no: str, visitados: set, componente: List[str]):
        """DFS para encontrar componente conectado"""
        visitados.add(no)
        componente.append(no)
        
        for vizinho in self.grafo[no]:
            if vizinho not in visitados:
                self._dfs_componente(vizinho, visitados, componente)
    
    def _identificar_pontos_criticos(self) -> List[str]:
        """Identifica pontos de articulação (pontos críticos)"""
        pontos_criticos = []
        componentes_originais = len(self._encontrar_componentes_conectados())
        
        # Cria cópia estática das chaves para iterar
        nos = list(self.grafo.keys())
        
        for no in nos:
            # Pula nós já removidos
            if no not in self.grafo:
                continue
                
            # Backup das conexões
            conexoes_backup = self.grafo[no].copy()
            vizinhos_backup = []
            
            # Remove o nó do grafo temporariamente
            for vizinho in conexoes_backup:
                if no in self.grafo[vizinho]:
                    vizinhos_backup.append(vizinho)
                    del self.grafo[vizinho][no]
            del self.grafo[no]
            
            # Verifica impacto na conectividade
            componentes_sem_no = len(self._encontrar_componentes_conectados())
            if componentes_sem_no > componentes_originais:
                pontos_criticos.append(no)
            
            # Restaura o nó
            self.grafo[no] = conexoes_backup
            for vizinho in vizinhos_backup:
                self.grafo[vizinho][no] = conexoes_backup[vizinho]
        
        return pontos_criticos
    def _gerar_relatorio_operacao(self):
        """Gera relatório completo da operação"""
        print("\n📋 RELATÓRIO FINAL DA OPERAÇÃO")
        print("=" * 50)
        
        # Análise da rede
        analise_rede = self.grafo_rede.analisar_conectividade_rede()
        
        # Status dos sensores
        sensores_ativos = self.lista_sensores.listar_sensores_por_status("ATIVO")
        
        # === CORREÇÃO: Relatório formatado de drones ===
        print("\n📊 RESUMO DE DRONES")
        print("-------------------")
        print("Drone      | Missões | Combustível | Água")
        print("-----------------------------------------")
        for drone_id, drone in self.drones_disponiveis.items():
            print(f"{drone_id} | {len(drone.historico_acoes)}       | {drone.nivel_combustivel}%     | {drone.nivel_agua}%")
        # === FIM DA CORREÇÃO ===

        # Hierarquia de zonas
        print(f"\n🌳 ESTRUTURA DE MONITORAMENTO:")
        self.arvore_zonas.visualizar_hierarquia_completa()
        
        print(f"\n📊 ESTATÍSTICAS FINAIS:")
        print(f"   Sensores ativos: {len(sensores_ativos)}")
        print(f"   Alertas processados: 3")
        print(f"   Drones utilizados: {len([d for d in self.drones_disponiveis.values() if d.historico_acoes])}")
        print(f"   Densidade da rede: {analise_rede['densidade_rede']:.2f}")
# ================================ SISTEMA PRINCIPAL - INTEGRAÇÃO ================================
class SistemaFireWatch:
    """
    Sistema principal que integra todas as estruturas de dados
    Simula operação completa de combate a queimadas
    """
    
    def __init__(self):
        # Inicializa todas as estruturas
        self.fila_sensores = FilaDadosSensores()
        self.heap_alertas = HeapAlertasCriticos()
        self.lista_sensores = ListaSensoresIoT()
        self.arvore_zonas = ArvoreZonasMonitoramento("Sudeste")
        self.grafo_rede = GrafoRedeSensores()
        
        # Gerenciamento de drones
        self.drones_disponiveis = {}
        self.missoes_ativas = {}
        
        # Configurações do sistema
        self.bases_drones = []
        self.equipes_emergencia = []
        
        print("🔥 SISTEMA FIREWATCH INICIALIZADO")
        print("=" * 50)
    
    def configurar_sistema_inicial(self):
        """Configura estrutura inicial do sistema"""
        # Adiciona estrutura hierárquica
        self.arvore_zonas.adicionar_estado("São Paulo", 248219.0)
        self.arvore_zonas.adicionar_estado("Minas Gerais", 586522.0)
        
        self.arvore_zonas.adicionar_municipio("São Paulo", "Campinas", 794.0)
        self.arvore_zonas.adicionar_municipio("São Paulo", "Sorocaba", 450.0)
        self.arvore_zonas.adicionar_municipio("Minas Gerais", "Uberlândia", 4115.0)
        
        self.arvore_zonas.adicionar_setor_monitoramento("São Paulo", "Campinas", "Setor Norte", 50.0)
        self.arvore_zonas.adicionar_setor_monitoramento("São Paulo", "Campinas", "Setor Sul", 45.0)
        
        # Configura rede de sensores no grafo
        self.grafo_rede.adicionar_no("BASE_CENTRAL", (-22.8191, -47.0677), "BASE")
        self.grafo_rede.adicionar_no("BASE_NORTE", (-22.7500, -47.1000), "BASE")
        
        self.grafo_rede.adicionar_no("SENSOR_001", (-22.8000, -47.0500), "SENSOR")
        self.grafo_rede.adicionar_no("SENSOR_002", (-22.8300, -47.0800), "SENSOR")
        self.grafo_rede.adicionar_no("SENSOR_003", (-22.7800, -47.1200), "SENSOR")
        
        self.grafo_rede.adicionar_no("ZONA_RISCO_A", (-22.8100, -47.0900), "POI")
        self.grafo_rede.adicionar_no("ZONA_RISCO_B", (-22.7700, -47.1100), "POI")
        
        # Conecta nós no grafo
        self.grafo_rede.conectar_nos("BASE_CENTRAL", "SENSOR_001")
        self.grafo_rede.conectar_nos("BASE_CENTRAL", "SENSOR_002")
        self.grafo_rede.conectar_nos("BASE_NORTE", "SENSOR_003")
        self.grafo_rede.conectar_nos("SENSOR_001", "ZONA_RISCO_A")
        self.grafo_rede.conectar_nos("SENSOR_002", "ZONA_RISCO_A")
        self.grafo_rede.conectar_nos("SENSOR_003", "ZONA_RISCO_B")
        self.grafo_rede.conectar_nos("BASE_CENTRAL", "BASE_NORTE")
        
        # Instala sensores na lista ligada
        self.lista_sensores.instalar_sensor("SEN_001", "MULTISENSOR", (-22.8000, -47.0500))
        self.lista_sensores.instalar_sensor("SEN_002", "TEMPERATURA", (-22.8300, -47.0800))
        self.lista_sensores.instalar_sensor("SEN_003", "FUMACA", (-22.7800, -47.1200))
        
        # Configura drones
        self.drones_disponiveis["DRONE_01"] = PilhaAcoesDrone("DRONE_01")
        self.drones_disponiveis["DRONE_02"] = PilhaAcoesDrone("DRONE_02")
        
        # Aloca drones às zonas
        self.arvore_zonas.alocar_drone_zona("São Paulo_Campinas", "DRONE_01")
        self.arvore_zonas.alocar_drone_zona("São Paulo_Sorocaba", "DRONE_02")
        
        print("✅ Configuração inicial concluída")
    
    def simular_operacao_completa(self):
        """Simula operação completa do sistema durante emergência"""
        print("\n🚨 INICIANDO SIMULAÇÃO DE EMERGÊNCIA")
        print("=" * 50)
        
        # 1. Sensores coletam dados e adicionam à fila
        self._simular_coleta_sensores()

        # Pequeno delay para processamento final
        import time
        time.sleep(5.0)
        
        # 2. Processa dados da fila e gera alertas no heap
        self._processar_dados_sensores()
        
        # 3. Processa alertas por prioridade
        self._processar_alertas_criticos()
        
        # 4. Coordena resposta com drones
        self._coordenar_resposta_drones()

        # Pequeno delay para processamento final
        import time
        time.sleep(5.5)
        
        # 5. Gera relatório final
        self._gerar_relatorio_operacao()
    
    def _simular_coleta_sensores(self):
        """Simula coleta de dados dos sensores IoT"""
        print("\n📡 COLETA DE DADOS DOS SENSORES")
        
        # Dados simulados de diferentes sensores
        dados_simulados = [
            ("SEN_001", 42.5, 25.0, 18.0, (-22.8000, -47.0500)),
            ("SEN_002", 38.0, 15.0, 35.0, (-22.8300, -47.0800)),
            ("SEN_003", 51.0, 45.0, 12.0, (-22.7800, -47.1200)),
            ("SEN_001", 48.5, 35.0, 15.0, (-22.8000, -47.0500)),  # Segunda leitura crítica
            ("SEN_002", 39.0, 18.0, 32.0, (-22.8300, -47.0800))
        ]
        
        for sensor_id, temp, fumaca, umidade, loc in dados_simulados:
            self.fila_sensores.adicionar_leitura_sensor(sensor_id, temp, fumaca, umidade, loc)
        
        print(f"📊 Total de leituras na fila: {self.fila_sensores.tamanho_fila()}")
    
    def _processar_dados_sensores(self):
        """Processa dados da fila de sensores"""
        print("\n⚙️ PROCESSAMENTO DE DADOS DOS SENSORES")
        
        while self.fila_sensores.tamanho_fila() > 0:
            leitura = self.fila_sensores.processar_proxima_leitura()
            if leitura:
                # Adiciona no heap de alertas
                self.heap_alertas.adicionar_alerta(leitura)
                
                # Atualiza status do sensor na lista ligada
                sensor = self.lista_sensores.buscar_sensor(leitura["sensor_id"])
                if sensor:
                    sensor.ultima_leitura = leitura
    
    def _processar_alertas_criticos(self):
        """Processa alertas por ordem de criticidade"""
        print("\n🚨 PROCESSAMENTO DE ALERTAS CRÍTICOS")
        
        alertas_processados = 0
        while alertas_processados < 3:  # Processa os 3 mais críticos
            alerta = self.heap_alertas.processar_alerta_mais_critico()
            if not alerta:
                break
                
            print(f"🔥 Processando alerta {alerta['criticidade']}: {alerta['sensor_id']}")
            
            # Atualiza nível de risco da zona
            if alerta["criticidade"] in ["CRÍTICO", "ALTO"]:
                self.arvore_zonas.atualizar_nivel_risco_zona("São Paulo_Campinas", alerta["criticidade"])
            
            alertas_processados += 1
    
    def _coordenar_resposta_drones(self):
        """Coordena resposta dos drones baseada nos alertas"""
        print("\n🚁 COORDENAÇÃO DE RESPOSTA DOS DRONES")
        
        # Simula emergência em zona de risco
        emergencia_local = "ZONA_RISCO_A"
        
        for drone_id, drone in self.drones_disponiveis.items():
            if drone.status_atual == "DISPONÍVEL":
                # Calcula rota ótima
                rota_info = self.grafo_rede.encontrar_rota_otima_drone("BASE_CENTRAL", emergencia_local)
                
                # Executa missão de reconhecimento
                drone.executar_missao("RECONHECIMENTO", (-22.8100, -47.0900))
                
                # Simula finalização da missão
                import time
                time.sleep(1)  # Simula tempo de missão
                drone.finalizar_missao_atual("SUCESSO", f"Rota seguida: {' → '.join(rota_info['rota_otima'])}")
                
                # Executa missão de combate se necessário
                dados_termicos = drone.obter_ultima_missao()["dados_termicos"]
                if dados_termicos.get("intensidade_fogo") in ["MÉDIA", "ALTA"]:
                    drone.executar_missao("COMBATE", (-22.8100, -47.0900), dados_termicos)
                    time.sleep(2)
                    drone.finalizar_missao_atual("SUCESSO", "Foco de incêndio controlado")
                
                break  # Usa apenas um drone para esta simulação
    
    def _gerar_relatorio_operacao(self):
        """Gera relatório completo da operação"""
        print("\n📋 RELATÓRIO FINAL DA OPERAÇÃO")
        print("=" * 50)
        
        # Análise da rede
        analise_rede = self.grafo_rede.analisar_conectividade_rede()
        
        # Status dos sensores
        sensores_ativos = self.lista_sensores.listar_sensores_por_status("ATIVO")
        
        # Histórico dos drones
        for drone_id, drone in self.drones_disponiveis.items():
            print(f"\n🚁 DRONE {drone_id}:")
            print(f"   Status: {drone.status_atual}")
            print(f"   Combustível: {drone.nivel_combustivel}%")
            print(f"   Água: {drone.nivel_agua}%")
            print(f"   Missões realizadas: {len(drone.historico_acoes)}")
            
            if drone.historico_acoes:
                ultima_missao = drone.obter_ultima_missao()
                print(f"   Última missão: {ultima_missao['tipo']} - {ultima_missao['resultado']}")
        
        # Hierarquia de zonas
        print(f"\n🌳 ESTRUTURA DE MONITORAMENTO:")
        self.arvore_zonas.visualizar_hierarquia_completa()
        
        print(f"\n📊 ESTATÍSTICAS FINAIS:")
        print(f"   Sensores ativos: {len(sensores_ativos)}")
        print(f"   Alertas processados: 3")
        print(f"   Drones utilizados: {len([d for d in self.drones_disponiveis.values() if d.historico_acoes])}")
        print(f"   Densidade da rede: {analise_rede['densidade_rede']:.2f}")

# ================================ EXECUÇÃO DO SISTEMA ================================
def main():
    """Função principal para executar o sistema FireWatch"""
    print("🔥 SISTEMA FIREWATCH - COMBATE INTELIGENTE A QUEIMADAS")
    print("🔥 Utilizando: Fila, Heap, Pilha, Lista Ligada, Árvore e Grafo")
    print("=" * 70)
    
    # Inicializa sistema
    firewatch = SistemaFireWatch()
    
    # Configura ambiente inicial
    firewatch.configurar_sistema_inicial()
    
    # Executa simulação completa
    firewatch.simular_operacao_completa()
    
    print("\n🎯 SISTEMA FIREWATCH CONCLUÍDO COM SUCESSO!")
    print("   Todas as estruturas de dados foram utilizadas:")
    print("   ✅ Fila (Queue) - Processamento de dados dos sensores")
    print("   ✅ Heap - Priorização de alertas críticos")
    print("   ✅ Pilha (Stack) - Histórico de ações dos drones")
    print("   ✅ Lista Ligada - Gerenciamento dinâmico de sensores")
    print("   ✅ Árvore - Hierarquia de zonas de monitoramento")
    print("   ✅ Grafo - Rede de sensores e otimização de rotas")

if __name__ == "__main__":
    main()

✅ Configuração concluída
🔥 SISTEMA FIREWATCH - COMBATE INTELIGENTE A QUEIMADAS
🔥 Utilizando: Fila, Heap, Pilha, Lista Ligada, Árvore e Grafo
🌳 Sistema de monitoramento iniciado para região: Sudeste
🔥 SISTEMA FIREWATCH INICIALIZADO
🏛️ Estado São Paulo adicionado (248219.0 km²)
🏛️ Estado Minas Gerais adicionado (586522.0 km²)
🏘️ Município Campinas adicionado ao estado São Paulo
🏘️ Município Sorocaba adicionado ao estado São Paulo
🏘️ Município Uberlândia adicionado ao estado Minas Gerais
🌲 Setor Setor Norte criado em Campinas
🌲 Setor Setor Sul criado em Campinas
🏠 Nó BASE_CENTRAL (BASE) adicionado em (-22.8191, -47.0677)
🏠 Nó BASE_NORTE (BASE) adicionado em (-22.75, -47.1)
📡 Nó SENSOR_001 (SENSOR) adicionado em (-22.8, -47.05)
📡 Nó SENSOR_002 (SENSOR) adicionado em (-22.83, -47.08)
📡 Nó SENSOR_003 (SENSOR) adicionado em (-22.78, -47.12)
📍 Nó ZONA_RISCO_A (POI) adicionado em (-22.81, -47.09)
📍 Nó ZONA_RISCO_B (POI) adicionado em (-22.77, -47.11)
🔗 Conexão: BASE_CENTRAL ↔ SENSOR_001 (peso: 